In [ ]:
!pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

# ユーザーからの入力を受け取り、パラメータを設定
num_nodes = int(input("ノード数を入力してください : "))  # ノード数
initial_infected_percentage = float(input("初期感染者の割合を入力してください (%): ")) / 100  # 初期感染者の割合
transmission_probability = float(input("感染の確率を入力してください (%): ")) / 100  # 感染の確率
recovery_probability = float(input("回復の確率を入力してください (%): ")) / 100  # 回復の確率
steps = int(input("シミュレーションのステップ数を入力してください (例: 30): "))  # シミュレーションのステップ数

# グラフの作成（ランダムな接続のあるエルデシュ-レーニグラフ）
G = nx.erdos_renyi_graph(n=num_nodes, p=0.1)

# 各ノードのステータス（S: Susceptible（未感染）, I: Infected（感染中）, R: Recovered（回復））
status = {node: 'S' for node in G.nodes()}

# 一部を感染状態にする
initial_infected_count = max(1, int(initial_infected_percentage * num_nodes))
initial_infected_nodes = random.sample(list(G.nodes()), initial_infected_count)
for node in initial_infected_nodes:
    status[node] = 'I'

def update_status(G, status, transmission_probability, recovery_probability):
    new_status = status.copy()
    for node in G.nodes():
        if status[node] == 'I':
            if random.random() < recovery_probability:
                new_status[node] = 'R'
            else:
                neighbors = list(G.neighbors(node))
                for neighbor in neighbors:
                    if status[neighbor] == 'S' and random.random() < transmission_probability:
                        new_status[neighbor] = 'I'
    return new_status

def draw_network(G, status, step):
    color_map = {'S': 'lightblue', 'I': 'red', 'R': 'green'}
    node_colors = [color_map[status[node]] for node in G.nodes()]
    plt.figure(figsize=(8, 8))
    nx.draw(G, node_color=node_colors, with_labels=True, node_size=300, font_size=8)
    plt.title(f'Step {step}')
    plt.show()

# 状態の履歴を保存するリスト
susceptible_count_history = []
infected_count_history = []
recovered_count_history = []

# シミュレーション開始
for step in range(steps):
    draw_network(G, status, step)
    # カウントと履歴の更新
    susceptible_count = sum(1 for state in status.values() if state == 'S')
    infected_count = sum(1 for state in status.values() if state == 'I')
    recovered_count = sum(1 for state in status.values() if state == 'R')

    susceptible_count_history.append(susceptible_count)
    infected_count_history.append(infected_count)
    recovered_count_history.append(recovered_count)

    # ステータスを更新
    status = update_status(G, status, transmission_probability, recovery_probability)

# 感染者・非感染者・回復者の数を表示する折れ線グラフ
plt.figure(figsize=(10, 6))
plt.plot(range(steps), susceptible_count_history, label="Susceptible", color='lightblue')
plt.plot(range(steps), infected_count_history, label="Infected", color='red')
plt.plot(range(steps), recovered_count_history, label="Recovered", color='green')
plt.xlabel('Step')
plt.ylabel('Count')
plt.title('SIR Model Simulation')
plt.legend()
plt.grid(True)
plt.show()